# Fine-tuning a model from questions and answers
In this notebook we take a the [human-eval-bia dataset](https://github.com/haesleinhuepf/human-eval-bia) to fine-tune OpenAI gpt4omni.

Read more about [fine tuning OpenAI models](https://platform.openai.com/docs/guides/fine-tuning).

In [1]:
import requests
import json
import pandas as pd
import time
import os
import openai
from sklearn.model_selection import train_test_split
from fine_tuning_utilities import load_jsonl_file, save_jsonl_file

In [2]:
url = "https://raw.githubusercontent.com/haesleinhuepf/human-eval-bia/main/data/human-eval-bia.jsonl"

response = requests.get(url)
data = [json.loads(line) for line in response.text.splitlines()]
df = pd.DataFrame(data)

df.head(2)

,task_id,prompt,canonical_solution,entry_point,test
0,../test_cases/apply_otsu_threshold_and_count_p...,def apply_otsu_threshold_and_count_postiive_pi...,\n import skimage\n import numpy as np\n...,apply_otsu_threshold_and_count_postiive_pixels,def check(candidate):\n import numpy as np\...
1,../test_cases/binary_closing.ipynb,"def binary_closing(binary_image, radius:int=1)...",\n import numpy as np\n import skimage\n...,binary_closing,def check(candidate):\n import numpy as np\...


First, we need to convert the data frame into jsonl format representing conversions between user and assistant as required by OpenAI.

In [3]:
all_data = []

for index, row in df.iterrows():
    messages_dict = {
        "messages": [
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['canonical_solution']}
        ]
    }
    all_data.append(messages_dict)

all_data[:2]

[{'messages': [{'role': 'user',
    'content': 'def apply_otsu_threshold_and_count_postiive_pixels(image):\n    """\n    Takes an image, applies Otsu\'s threshold method to it to create a binary image and \n    counts the positive pixels.\n    """'},
   {'role': 'assistant',
    'content': '\n    import skimage\n    import numpy as np\n    binary_image = image > skimage.filters.threshold_otsu(image)\n\n    result = np.sum(binary_image)\n\n    return result'}]},
 {'messages': [{'role': 'user',
    'content': 'def binary_closing(binary_image, radius:int=1):\n    """\n    Applies binary closing to a binary_image with a square footprint with a given radius.\n    """'},
   {'role': 'assistant',
    'content': '\n    import numpy as np\n    import skimage\n    size = radius * 2 + 1\n    return skimage.morphology.binary_closing(binary_image, footprint=np.ones((size, size)))'}]}]

In [4]:
training_data, validation_data = train_test_split(all_data, test_size=0.2, random_state=42)

len(training_data), len(validation_data)

(45, 12)

In [5]:
# save training data to a temporary file
training_data_file_path = "training_data.jsonl"
save_jsonl_file(training_data, training_data_file_path)

validation_data_file_path = "validation_data.jsonl"
save_jsonl_file(validation_data, validation_data_file_path)

We just initialize a client to the OpenAI API.

In [6]:
client = openai.OpenAI()

... and upload files:

In [7]:
training_file = client.files.create(
    file=open(training_data_file_path, "rb"),
    purpose='fine-tune',
)
validation_file = client.files.create(
    file=open(validation_data_file_path, "rb"),
    purpose='fine-tune',
)

# wait until preprocessing is finished
while client.files.retrieve(training_file.id).status != "processed" or client.files.retrieve(validation_file.id).status != "processed":
    time.sleep(30)

print("Uploading / preprocessing done.")

Uploading / preprocessing done.


In [8]:
# start fine-tuning
fine_tuning_job = client.fine_tuning.jobs.create(
                        training_file=training_file.id, 
                        validation_file=validation_file.id,
                        model="gpt-4o-2024-08-06")
fine_tuning_job

FineTuningJob(id='ftjob-vtKSrOY5233kun4fkBYjlQXz', created_at=1724232888, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-0POmhzyaeDng5lZtM7Cls3vt', result_files=[], seed=59560052, status='validating_files', trained_tokens=None, training_file='file-xGVvsjSuhLPq1JHCGTJaKJHS', validation_file='file-TCGaIoxNChEdyoItiN6O4CzV', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [9]:
job_details = client.fine_tuning.jobs.retrieve(
                        fine_tuning_job.id)
job_details

FineTuningJob(id='ftjob-vtKSrOY5233kun4fkBYjlQXz', created_at=1724232888, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-0POmhzyaeDng5lZtM7Cls3vt', result_files=[], seed=59560052, status='validating_files', trained_tokens=None, training_file='file-xGVvsjSuhLPq1JHCGTJaKJHS', validation_file='file-TCGaIoxNChEdyoItiN6O4CzV', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [10]:
job_details.status

'validating_files'

In [11]:
job_details = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
job_details.status

'validating_files'

In [12]:
job_details = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
job_details.error

Error(code=None, message=None, param=None)

In case you don't want to run the cell above repeatedly manually, one can also run such a request in a loop:

In [13]:
while client.fine_tuning.jobs.retrieve(fine_tuning_job.id).status not in ["succeeded", "failed"]:
    time.sleep(120)

job_details = client.fine_tuning.jobs.retrieve(
                fine_tuning_job.id)
job_details.status 

'succeeded'

In [14]:
job_details.error

Error(code=None, message=None, param=None)

## Retrieving the new model name
Once done, one can retrieve the name of the fine-tuned model like this:

In [15]:
model_name = job_details.fine_tuned_model
model_name

'ft:gpt-4o-2024-08-06:leipzig-university::9ydjNWWH'

Final remark: Running this fine-tuning notebook costed about $0.27.